# Introduction 
You just accepted a job at CausalMarketing, a firm that offers marketing consulting services. 
For your first assignment, you've been tasked with helping a large online clothing store, Nosara, with their targeted advertising campaign.


Nosara is promoting a new handbag model priced at ***$100***, which is not yet available for sale. 

They randomly selected 3 million past customers and sent them emails offering the opportunity to pre-order the handbag. 

These customers were divided into two groups: 
***one received only the pre-order invitation (control group)***, and 
***the other received a 20% discount if they pre-ordered (treatment group)***. 
Each email allows the purchase of ***a single handbag***.

Nosara has ***10,979,592*** past customers who haven't been emailed yet. 
They want you to help them design a data-driven solution to 
decide which of these customers should ***receive a discount email*** and which should ***receive a regular email*** to ***maximize*** sales after discounts.

Nosara has shared two datasets:

one of customers who have already been emailed (nosara_labeled) and another of customers not yet emailed (nosara_unlabeled). The fields are:

Features (all numeric): f0, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11

treatment: Indicates group (1 = treated, 0 = control). 

preorder: Indicates if the customer pre-ordered (1=Yes, 0=No).

Note: Feature names were anonymized, and their values randomly projected to keep their predictive power without risking the privacy of Nosara’s customer base.

## Analysis

#### Business understanding 

Need to help the company to decide who need to receive the promotion emails to maximum the profits 
so the Benefit > cost 

#### Cost and Benefit martix 

|    | pre-order (1) | do not pre-order (0) |
| -------- | ------- | ------ |
| regular email   | BC = 100| 0 |
| 20% discount email| BC - DC = 80 | 0 |

Bag Cost(BC) : 100

Discount Cost(DC) : 20

#### Decision need to make 
10,979,592

#### what possbility we have 

|    | pre-order (1) | do not pre-order (0) |
| -------- | ------- | ------ |
| regular email (control)  | P(1\|RE) | P(0\|RE) |
| 20% discount email (treatment) | P(1\|DE) | P(0\|DE) |

P(1|RE) = from control group

P(0|RE) = from control group

P(1|DE) = from treatment group

P(0|DE) = from treatment group

#### Understand the business 

Nothing . just know it selling the bag witgh pre-order and want to get the maximum value

#### Understand the data 

2 datasets and group 

##### Group:

***one received only the pre-order invitation (control group)***  

***the other received a 20% discount if they pre-ordered (treatment group)***. 

##### Data: 

***receive a discount email***  (nosara_labeled)

***receive a regular email***   (nosara_unlabeled)

#### Modeling
Nth to do modeling is provided 



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

# Load Data
df_labeled = pd.read_csv("nosara_labeled.csv")
df_unlabeled = pd.read_csv("nosara_unlabeled.csv")

# Pre-process data
feature_names = ['f0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11']
X = df_labeled[feature_names]
y = df_labeled.preorder
t = df_labeled.treatment
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
X_unlabeled = scaler.transform(df_unlabeled[feature_names])

# Split data
X_tr, X_te, y_tr, y_te, t_tr, t_te = train_test_split(X, y, t, test_size=0.2, random_state=1337)

# Data is large, so be patient with the training. 
# Model for treated
model_treat = LogisticRegression().fit(X_tr[t_tr==1],y_tr[t_tr==1])
# Model for control
model_control = LogisticRegression().fit(X_tr[t_tr==0],y_tr[t_tr==0])
print("Done!")

Done!


## calculate the prob. on 

|    | pre-order (1) | do not pre-order (0) |
| -------- | ------- | ------ |
| regular email (control)  | P(1\|RE) | P(0\|RE) |

In [63]:
# output the prediction in treatment
treat_predict=model_treat.predict(X_te[t_te==1])


## calculate the prob. on 

|    | pre-order (1) | do not pre-order (0) |
| -------- | ------- | ------ |
| 20% discount email (treatment) | P(1\|DE) | P(0\|DE) |

In [76]:
from sklearn.metrics import confusion_matrix
cm_treat = confusion_matrix(treat_predict, y_te[t_te==1])
print("test case size on E(1|RE) = {}".format(treat_predict.shape[0]))
print("confusion matrix is :")
print("[[TN, FP],")
print("[FN, TP]]")
print(cm_treat)
print("so the P(1|DE) = (E(1|FP,DE) + E(1|TP,DE))/E(1|DE)")
P_1_DE = (cm_treat[0,1]+cm_treat[1,1])/treat_predict.shape[0]
print(f"P(1|DE)={P_1_DE}")
print("so the P(0|RE) = (E(TN|0|RE) + E(FN|0|RE))/E(1|RE)")
P_0_DE = (cm_treat[0,0]+cm_treat[1,0])/treat_predict.shape[0]
print(f"P(0|DE)={P_0_DE}")


test case size on E(1|RE) = 509834
confusion matrix is :
[[TN, FP],
[FN, TP]]
[[480075  16702]
 [  5273   7784]]
so the P(1|DE) = (E(1|FP,DE) + E(1|TP,DE))/E(1|DE)
P(1|DE)=0.04802739715279875
so the P(0|RE) = (E(TN|0|RE) + E(FN|0|RE))/E(1|RE)
P(0|DE)=0.9519726028472012


In [78]:
# Effect 
effect = P_1_DE * 80 - P_0_DE * 0
print(f"Effect={effect}")
# prob pn P(1)
print(f"E(1) = E(1|RE)+E(1|DE)")
E_1_DE = cm_treat[0,1]+cm_treat[1,1]
E_1 = E_1_DE 
print(f"# of events sending discount email={E_1}")
print(f"# of events sending email={X_te.shape[0]}")
P_1 = E_1/X_te.shape[0]
print(f"P(send discount email)={P_1}")
# Effect of Decision Rule 
effect_decision_rule = 10979592*P_1*effect
print(f"Effect Decision_Rule={effect_decision_rule}")

Effect=3.8421917722239
E(1) = E(1|RE)+E(1|DE)
# of events sending discount email=24486
# of events sending email=600000
P(send discount email)=0.04081
Effect Decision_Rule=1721598.3372072822
